# Part 2: Basic LLM Chat Tool

## Introduction

In this part, you'll create a simple command-line chat tool that interacts with a Large Language Model (LLM) through the Hugging Face API. This tool will allow you to have conversations with an LLM about healthcare topics.

## Learning Objectives

- Connect to the Hugging Face API
- Create a basic interactive chat loop
- Handle simple error cases
- Test with healthcare questions

## Setup and Installation

In [19]:
import os
import sys
import requests
import time
import logging
import argparse
from pathlib import Path
from typing import Optional, List, Dict
import openai
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger("llm_test_logger")

project_root = Path.cwd()

utils_dir = project_root / "utils"
results_dir = project_root / "results" / "part_2"

utils_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)



Project root directory: /Users/hteshome/Desktop/7-transformers-haile-teshome


## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [ ]:

openai.api_key = os.getenv("OPENAI_API_KEY")

def query(message, model="gpt-3.5-turbo"):
    """
    Send a prompt to the OpenAI ChatGPT API.

    Args:
        message (str): A string containing the user's input.
        model (str): The model to query (default: gpt-3.5-turbo).

    Returns:
        str: The model's response.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": message}]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"[Error] {str(e)}"

# Interactive test with graceful exit
if __name__ == "__main__":
    print("ChatGPT Chat — type 'exit' to quit.")

    try:
        while True:
            try:
                user_input = input("\nYou: ")
                if user_input.lower() == "exit":
                    print("Goodbye!")
                    break

                response = query(user_input)
                print(f"ChatGPT: {response}")

            except EOFError:
                print("\n[Session ended with Ctrl+D]")
                break

    except KeyboardInterrupt:
        print("\n[Interrupted with Ctrl+C — exiting]")


## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [ ]:


def query(message, model="gpt-3.5-turbo"):
    """
    Send a prompt to the OpenAI ChatGPT API.

    Args:
        message (str): A string containing the user's input.
        model (str): The model to query (default: gpt-3.5-turbo).

    Returns:
        str: The model's response.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": message}]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"[Error] {str(e)}"

if __name__ == "__main__":
    print("ChatGPT Test — type 'exit' to quit.")

    try:
        while True:
            try:
                user_input = input("\nYou: ")
                if user_input.lower() == "exit":
                    print("Goodbye!")
                    break

                response = query(user_input)
                print(f"ChatGPT: {response}")

            except EOFError:
                print("\n[Session ended with Ctrl+D]")
                break

    except KeyboardInterrupt:
        print("\n[Interrupted with Ctrl+C — exiting]")


### Contextual Conversation Script

Create a script that maintains conversation history:

In [12]:

def get_response(prompt, history=None, model_name="gpt-3.5-turbo", history_length=3):
    """
    Get a response from the OpenAI model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Model to use (default: gpt-3.5-turbo)
        history_length: How many past interactions to include
        
    Returns:
        response_text: The model's response
    """
    if history is None:
        history = []

    # Build messages with past history 
    messages = []
    for user, assistant in history[-history_length:]:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})

    # Add the current prompt
    messages.append({"role": "user", "content": prompt})

    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=messages
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        return f"[ERROR] {e}"

def run_chat(model_name="gpt-3.5-turbo", history_length=3):
    """
    Run an interactive contextual chat session
    """
    print("Contextual LLM Chat\nType 'exit' to quit.")
    history = []

    try:
        while True:
            try:
                user_input = input("\nYou: ")
            except EOFError:
                print("\n[EOF received] Exiting...")
                break

            if user_input.lower().strip() == 'exit':
                print("Goodbye!")
                break

            response = get_response(user_input, history, model_name, history_length)
            print(f"ChatGPT: {response}")

            # Append current interaction to history
            history.append((user_input, response))

    except KeyboardInterrupt:
        print("\n[Interrupted] Exiting...")

if __name__ == "__main__":
    run_chat()


Welcome to the Contextual LLM Chat! Type 'exit' to quit.
ChatGPT: I am here to assist you with any questions or tasks you may have. Just let me know how I can help.
Goodbye!


## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.

In [32]:
import os
import sys
from pathlib import Path

sys.path.append(str(Path.cwd() / "utils"))

from one_off_chat import query as get_one_off_response

def test_chat(questions, model_name="gpt-3.5-turbo", api_key=None):
    if api_key:
        import openai
        openai.api_key = api_key

    results = {}
    for question in questions:
        print(f"Testing: {question}")
        response = get_one_off_response(message=question, model=model_name)
        results[question] = response
    return results

test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="/Users/hteshome/Desktop/7-transformers-haile-teshome/results/part_2/example.txt"):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("# LLM Chat Tool Test Results\n\n")
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("python /Users/hteshome/Desktop/7-transformers-haile-teshome/utils/one_off_chat.py --api_key YOUR_KEY\n")
        f.write("python /Users/hteshome/Desktop/7-transformers-haile-teshome/utils/conversation.py --api_key YOUR_KEY\n")
        f.write("```\n\n")
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        for question, response in results.items():
            q = question.replace(",", " ").replace("\n", " ").strip()
            r = response.replace(",", " ").replace("\n", " ").strip()
            f.write(f"{q},{r}\n")
        f.write("```\n")

if __name__ == "__main__":
    api_key = os.getenv("OPENAI_API_KEY") or input("Enter your OpenAI API key: ").strip()
    results = test_chat(test_questions, api_key=api_key)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")


Testing: What are the symptoms of gout?
Testing: How is gout diagnosed?
Testing: What treatments are available for gout?
Testing: What lifestyle changes can help manage gout?
Testing: What foods should be avoided with gout?
Test results saved to results/part_2/example.txt


## Progress Checkpoints

1. **API Connection**:
   - [ ] Successfully connect to the Hugging Face API
   - [ ] Send a query and receive a response
   - [ ] Handle API errors gracefully

2. **Chat Function Implementation**:
   - [ ] Implement the get_response function
   - [ ] Create the run_chat function for interactive sessions
   - [ ] Handle errors and edge cases

3. **Command Line Interface**:
   - [ ] Create a parser with appropriate arguments
   - [ ] Implement the main function
   - [ ] Test the CLI functionality

4. **Testing and Evaluation**:
   - [ ] Test the functions with healthcare questions
   - [ ] Save the results in a structured format
   - [ ] Analyze the quality of responses

## Common Issues and Solutions

1. **API Access Issues**:
   - Problem: Rate limiting
   - Solution: Implement exponential backoff and retry logic
   - Problem: Authentication errors
   - Solution: Verify API key and environment variables

2. **Response Parsing Issues**:
   - Problem: Unexpected response format
   - Solution: Add error handling for different response structures
   - Problem: Empty or error responses
   - Solution: Provide meaningful fallback responses

3. **CLI Issues**:
   - Problem: Arguments not parsed correctly
   - Solution: Test with different argument combinations
   - Problem: Script not executable
   - Solution: Check file permissions

## What to Submit

1. Your implementation of the chat scripts:
   - Basic requirement: `utils/one_off_chat.py` for single prompt/response chat
   - Stretch goal (optional): `utils/conversation.py` for contextual chat
   - Testing script: `utils/test_chat.py` to evaluate your implementation

2. Test results in `results/part_2/example.txt` with the following format:
   - Usage examples section showing how to run your scripts
   - Test results section with CSV-formatted question/response pairs
   - If you implemented the stretch goal, include examples of contextual exchanges

The auto-grader should check:
1. That your chat scripts can be executed
2. That they correctly handle the test questions
3. That your results file contains the required sections